# Derivative Dynamic Time Warping 

##### Loading Packages

In [1]:
# Boto Package
import boto3
import json

# Graphing Packages
import matplotlib.pyplot as plt
plt.style.use('bmh')

# Numpy
import numpy as np

# Pandas
import pandas as pd

# Datetime
import datetime
import pytz


In [2]:
tz = pytz.timezone('Asia/Singapore')

In [3]:
def acc_clean(df_a):
    acc = df_a.copy()
    acc.reset_index(drop=True, inplace=True)

    # Processing Timestamp
    acc['Timestamp'] = pd.to_numeric(acc['Timestamp'])

    # Convert Timestamp
    from datetime import datetime
    # note that timeline is in miliseconds
    # divide by 1000 before converting
    acc['Timestamp'] = acc['Timestamp'].astype(float)
    acc['Timestamps'] = acc['Timestamp'].astype(float)/1000
    acc['Time'] = acc['Timestamps'].apply(lambda x : datetime.fromtimestamp(x,tz))
    acc['Time'] = acc['Time'].dt.strftime("%Y-%m-%d %H:%M:%S")
 
 
    acc.drop(['Timestamps'], axis=1, inplace=True)
    acc['X'] = acc['X'].astype(float)
    acc['Y'] = acc['Y'].astype(float)
    acc['Z'] = acc['Z'].astype(float)

    # Calculating the total acceleration with the formula
    acc['X2'] = acc['X'].astype(float).apply(np.square)
    acc['Y2'] = acc['Y'].astype(float).apply(np.square)
    acc['Z2'] = acc['Z'].astype(float).apply(np.square)

    # Calculating acceleration 
    acc['Total Acceleration'] = np.sqrt(acc['X2'] + acc['Y2'] + acc['Z2'])
    pd.to_numeric(acc['Total Acceleration'])

    # Drop X2, Y2, Z2
    acc.drop(['X2'], axis = 1, inplace = True)
    acc.drop(['Y2'], axis = 1, inplace = True)
    acc.drop(['Z2'], axis = 1, inplace = True)

    # Converting X, Y and Z to numeric
    acc['X'] =  round(pd.to_numeric(acc['X']),6)
    acc['Y'] =  round(pd.to_numeric(acc['Y']),6)
    acc['Z'] =  round(pd.to_numeric(acc['Z']),6)

    return acc

### Derivative Dynamic Time Warping (DDTW)

#### With DDTW, we need to create higher level features from the signal obtained

In [5]:
import scipy

In [6]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,zero_one_loss

import time

In [7]:
# Standardisation of the two time series
# Normalisation of the data
# Creating higher level features with the equation given in the paper

from scipy import stats

def ddtw_preprocessing_norm1(ac):
    xf = ac['Xf'].values
    yf = ac['Yf'].values
    zf = ac['Zf'].values

    time = ac['Time'].values
    timestamps = ac['Timestamp'].values
    timestamp = timestamps
    filteracc = np.array(ac[['filteracceleration']])

    cleanacc = []
    for sublist in filteracc:
        for item in sublist:
            cleanacc.append(item)
    
    cleanacc = stats.zscore(cleanacc)

    ###### Done with normalisation of filtered total acceleration#####
    
    ##### transforming original points to higher-level function for DDTW #####    

    ac_n = pd.DataFrame({'time':time, 'timestamp':timestamp, 'xf':xf, 'yf':yf, 'zf':zf, 'cleanacc': cleanacc})
    
    ac_n['cleanacclag1'] = ac_n.cleanacc.shift(1, fill_value = 0)
    
    ac_n['cleanaccplus1'] = ac_n.cleanacc.shift(-1,fill_value = 0)
    
    ac_n['cleanh'] = ac_n['cleanacc'] - ac_n['cleanacclag1'] + ((ac_n['cleanaccplus1'] + ac_n['cleanacclag1'])/2) # transforming to higher-level features
    
    ac_n['cleanh'] = ac_n['cleanh']/2
    
    return ac_n

## Hard-coding DTW

#### Package does not work well on Python so we hard-coded the dynamic time warping process. 

#### Note: Codes are specific to this particular project that I did, Users should tweak codes to their context. 

#### I was working on smartphone sensor data that is triaxial. The 'dnorm' is a score for the DDTW/ DTW.

#### DDTW is essentially DTW on the higher level features obtained from the above code.

#### The more similar two signals are, the lower the dnorm will be.

In [29]:
## Function to obtain the normalised DTW score with two series of cleaned and filtered data in a numpy series format
def dtw_modeldist1(series1,series2):
     
    # creating the m x n matrix
    i = len(series1)
    j = len(series2)

    # calculating distance of warping path
    def dtw_dist(s, t):
        n, m = len(s), len(t)
        dtw_matrix = np.zeros((n+1, m+1))
        for i in range(n+1):
            for j in range(m+1):
                dtw_matrix[i, j] = np.inf
        dtw_matrix[0, 0] = 0
    
        for i in range(1, n+1):
            for j in range(1, m+1):
                cost = np.square(s[i-1] - t[j-1])
                # take last min from a square box
                last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
                dtw_matrix[i, j] = cost + last_min
        return dtw_matrix

    dtw_dist = dtw_dist(series1,series2)
    distance = dtw_dist[i,j] # top right element is the accumulated distance matrix
    
    # traceback
    
    def _traceback(D):
        i, j = np.array(D.shape) - 2
        p, q = [i], [j]
        while (i > 0) or (j > 0):
            tb = np.argmin((D[i, j], D[i, j + 1], D[i + 1, j]))
            if tb == 0:
                i -= 1
                j -= 1
            elif tb == 1:
                i -= 1
            else:  # (tb == 2):
                j -= 1
            p.insert(0, i)
            q.insert(0, j)
        return np.array(p), np.array(q)

    path = _traceback(dtw_dist)

    #processing to get the dnorm
    ilist = path[0].tolist()
    jlist = path[1].tolist()

    pathdist = []

    for i in range(len(path[0])):
        il = ilist[i]
        jl = jlist[i]
        t = float(dtw_dist[il][jl])
        pathdist.append(t)
    
    from numpy import inf
    for i in range(len(pathdist)):
        if pathdist[i] == inf:
            pathdist[i] = 0.0

    pathdistsum = sum(pathdist)

    # normalisation 
    def normdtw(path,pathdistsum):
        dnorm = pathdistsum/(i*j) *(len(path[0]))
        return dnorm
    
    dnorm = normdtw(path,distance)
   
    return distance, dtw_dist, path, dnorm